<a href="https://colab.research.google.com/github/shivamnegi92/GenAI-Playlist/blob/main/Agents/Agentic_Deep_Researcher_HF_Mistral_Langraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate huggingface_hub langchain duckduckgo-search langchain-community langgraph  auto-gptq optimum langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883

In [2]:
# # pip install streamlit langgraph duckduckgo-search langchain openai
# !pip install langchain-community

In [3]:
# # pip install auto-gptq optimum
# !pip install optimum

In [ ]:
import time
from typing import Dict, Any, List, TypedDict

import torch
from duckduckgo_search import DDGS
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain.tools import tool
from langchain.schema import HumanMessage, AIMessage
from langchain.schema.runnable import Runnable
from transformers import AutoTokenizer, AutoModelForCausalLM


# ----------- Web Search Tool -----------
@tool
def web_search(query: str) -> str:
    """Search the web for recent info about a topic using DuckDuckGo."""
    results = DDGS().text(query, max_results=3)
    summaries = [f"{r['title']}: {r['body'][:150]}... ({r['href']})" for r in results]
    return "\n".join(summaries)


# ----------- HF Chat Model Wrapper -----------
class HFChatModel(Runnable):
    def __init__(self):
        model_id = "mistralai/Mistral-7B-Instruct-v0.2"
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            torch_dtype=torch.float16,
            trust_remote_code=True
        )

    def invoke(self, messages, config=None):
        prompt = "\n".join([
            f"User: {msg.content}" if isinstance(msg, HumanMessage)
            else f"Assistant: {msg.content}"
            for msg in messages
        ])
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7,
                top_k=40,
                top_p=0.9
            )
        decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        reply = decoded[len(prompt):].strip().split("Assistant:")[-1].strip()
        return AIMessage(content=reply)


llm = HFChatModel()


# ----------- LangGraph Node Functions -----------
def planner_node(state: Dict[str, Any]) -> Dict[str, Any]:
    messages = state["messages"]
    prompt = "List exactly 2 short steps to research this topic. Keep each step concise (1 sentence)."
    print(f"\n[🧠 Planner Prompt]: {prompt}")
    response = llm.invoke(messages + [HumanMessage(content=prompt)])
    print(f"[📋 Planner Output]: {response.content[:300]}")
    return {"messages": messages + [response]}


def researcher_node(state: Dict[str, Any]) -> Dict[str, Any]:
    messages = state["messages"]
    last_instruction = messages[-1].content
    query = last_instruction.split("\n")[0].strip("-•123. ")
    print(f"\n[🔍 Researcher running web_search on]: {query}")
    result = web_search.invoke(query)
    print(f"[📄 Research Result Snippet]:\n{result[:300]}...\n")
    return {"messages": add_messages(messages, HumanMessage(content=f"Web result for '{query}': {result}"))}


def writer_node(state: Dict[str, Any]) -> Dict[str, Any]:
    messages = state["messages"]
    prompt = "Write a 1-paragraph summary based on the research above. Focus only on key findings."
    print(f"\n[✍️ Writer Prompt]: {prompt}")
    response = llm.invoke(messages + [HumanMessage(content=prompt)])
    print(f"[📝 Final Report Preview]: {response.content[:300]}")
    return {"messages": messages + [response]}


# ----------- LangGraph Definition -----------
class GraphState(TypedDict):
    messages: List[Any]

workflow = StateGraph(GraphState)
workflow.add_node("Planner", planner_node)
workflow.add_node("Researcher", researcher_node)
workflow.add_node("Writer", writer_node)
workflow.set_entry_point("Planner")
workflow.add_edge("Planner", "Researcher")
workflow.add_edge("Researcher", "Writer")
workflow.set_finish_point("Writer")
graph = workflow.compile()


# ----------- Get Topic Safely -----------
try:
    import sys
    if len(sys.argv) > 1 and not sys.argv[1].startswith("-"):
        topic = sys.argv[1]
    else:
        topic = input("Enter your research topic: ").strip() or "Impacts of generative AI in healthcare"
except:
    topic = input("Enter your research topic: ").strip() or "Impacts of generative AI in healthcare"


# ----------- Run the Agent -----------
print(f"\n🚀 Starting research on: {topic}\n")

state = {"messages": [HumanMessage(content=f"Research topic: {topic}")]}
final_state = graph.invoke(state)

final_report = final_state["messages"][-1].content
print("\n✅ DONE! Final Report:\n")
print(final_report)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]